# Preparando o ambiente
Bibliotecas e inicializações necessárias

In [50]:
from sqlalchemy import create_engine
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import os

import torch
import torch.nn as nn
import torch.functional as F 
import torch.optim as optim


load_dotenv()

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
    .format(user=os.environ['DB_USER'],
            host=os.environ['DB_HOST'],
            pw=os.environ['DB_PASSWORD'],
            db=os.environ['DB_DATABASE']))


C:\Users\lhlag\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Carregamento dos dados
Nesse primeiro passo é feito o carregamento das ações que vamos usar para treinar a ia

In [ ]:
# Primeiro carregar os dados relativos as entradas que vamos treinar a ia

# Nesse software foi escolhido para treinar a ia usando ações do setor elétrico



acoes = [
    "AESB3",
    "ALUP11",
    "COCE5",
    "CMIG4",
    "CPLE6",
    "CPFE3",
    "ELET3",
    "ENBR3",
    "ENGI11",
    "ENEV3",
    "EGIE3",
    "EQTL3",
    "LIGT3",
    "NEOE3",
    "MEGA3",
    "TAEE11",
    "TRPL4",
]

dados_acoes = pd.read_sql('SELECT * FROM acao_historico_inputs WHERE stock_name IN (\''+'\',\''.join(acoes)+'\')',con=engine)

dados_acoes.drop('id',inplace=True,axis=1)
dados_acoes.drop('stock_name',inplace=True,axis=1)
dados_acoes.drop('trading_date',inplace=True,axis=1)
dados_acoes.drop('period_variation_open',inplace=True,axis=1)
dados_acoes.drop('period_variation_close',inplace=True,axis=1)


In [64]:
dados_acoes

,day1_price_open,day1_price_min,day1_price_max,day1_price_avg,day1_price_close,day2_price_open,day2_price_min,day2_price_max,day2_price_avg,day2_price_close,...,day9_price_min,day9_price_max,day9_price_avg,day9_price_close,global_price_open,global_price_min,global_price_max,global_price_avg,global_price_close,period_variation_avg
0,15.94,15.62,15.65,15.70,15.62,16.20,15.94,15.98,16.01,15.94,...,16.90,16.93,17.01,16.92,0.0,15.62,15.80,16.62,0.0,-0.25
1,15.60,15.77,15.80,15.87,15.77,15.94,15.62,15.65,15.70,15.62,...,17.67,17.69,17.41,17.69,0.0,15.51,15.53,16.54,0.0,-2.21
2,15.84,15.51,15.53,15.66,15.51,15.60,15.77,15.80,15.87,15.77,...,17.40,17.50,17.50,17.40,0.0,15.49,15.51,16.45,0.0,-1.28
3,15.52,15.49,15.51,15.52,15.49,15.84,15.51,15.53,15.66,15.51,...,17.00,17.03,17.04,17.00,0.0,15.35,15.36,16.38,0.0,0.84
4,15.58,15.35,15.36,15.46,15.35,15.52,15.49,15.51,15.52,15.49,...,16.75,16.80,16.84,16.75,0.0,15.35,15.85,16.32,0.0,2.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23405,23.25,23.30,23.31,23.28,23.31,23.18,23.25,23.26,23.29,23.25,...,23.26,23.28,23.33,23.28,0.0,17.41,23.13,43.67,0.0,-1.72
23406,23.12,23.08,23.13,23.11,23.13,23.25,23.30,23.31,23.28,23.31,...,22.85,22.86,22.94,22.86,0.0,17.41,22.89,43.66,0.0,-0.74
23407,23.12,22.88,22.89,22.88,22.89,23.12,23.08,23.13,23.11,23.13,...,22.71,22.74,22.75,22.74,0.0,17.41,22.91,43.64,0.0,-0.09
23408,22.95,22.89,22.91,22.94,22.91,23.12,22.88,22.89,22.88,22.89,...,22.91,22.92,22.71,22.92,0.0,17.41,22.89,43.63,0.0,0.00


In [56]:
Y = dados_acoes['period_variation_avg'].tolist()
X = dados_acoes.drop('period_variation_avg',axis=1).values.tolist()


'''
    0 - se for menor de -3%
    1 - se for menor que -1.5%
    2 - se for menor que -0.5%
    3 - se for menor que 0.5%
    4 - se for menor que 1.5%
    5 - se for menor que 3%
'''

separator = [-3,-1.5,-0.5,0.5,1.5,3,float('inf')]
a = np.array([0,1,2,3,4,5,6])
for idx,y_t in enumerate(Y):
    index = next(x[0] for x in enumerate(separator) if x[1] > y_t)
    Y[idx]= (a==index).astype('float')

for idx,x_t in enumerate(X):
    X[idx] = np.reshape(x_t,(10,5))
X = np.array(X)
Y = np.array(Y)

    

# Embaralhamento e Separação dos Dados

In [60]:
def split_and_shuffle(X, Y, perc = 0.1):
  ''' Esta função embaralha os pares de entradas
      e saídas desejadas, e separa os dados de
      treinamento e validação
  '''
  # Total de amostras
  tot = len(X)
  # Emabaralhamento dos índices
  indexes = np.arange(tot)
  np.random.shuffle(indexes)
  # Calculo da quantidade de amostras de
  # treinamento
  n = int((1 - perc)*tot)
  Xt = X[indexes[:n]]
  Yt = Y[indexes[:n]]
  Xv = X[indexes[n:]]
  Yv = Y[indexes[n:]]
  return Xt, Yt, Xv, Yv

Xt,Yt,Xv,Yv = split_and_shuffle(X,Y,0.15)

In [61]:
Xt = torch.from_numpy(Xt)
Yt = torch.from_numpy(Yt)
Xv = torch.from_numpy(Xv)
Yv = torch.from_numpy(Yv)

Xt = Xt.unsqueeze(1)
Xv = Xv.unsqueeze(1)

In [62]:
print('Dados de treinamento:')
print('Xt', Xt.size(), 'Yt', Yt.size())
print()
print('Dados de validação:')
print('Xv', Xv.size(), 'Yv', Yv.size())

Dados de treinamento:
Xt torch.Size([19898, 1, 10, 5]) Yt torch.Size([19898, 7])

Dados de validação:
Xv torch.Size([3512, 1, 10, 5]) Yv torch.Size([3512, 7])


# Projeção da Rede Neural

1.   `conv1`: Camada convolucional com kernel 3x1, 3 canais de saída, sem padding, stride 1 e ativação ReLU.
2.   `pool1`: Camada _max-pooling_ 3x1, com stride 1.
3.   `conv2`: Camada convolucional com kernel 3x2, 5 canais de saída, sem padding, stride 1 e ativação ReLU.
4.   `drop1`: Dropout 10%.
5.   `lin1`: Camada feedforward que recebe os dados serializados e gera as saídas. A função de ativação final é _softmax_, mas ela é implementada no cálculo da função de custo, então não precisa ser considerada aqui.

N1 = 3

N2a = 8

N2b = 5

N3 = 3 * 8 * 5

N4 = 3

N5a = 6

N5b = 5

N6 = 3 * 6 * 5

N7 = 5

N8a = 4

N8b = 4

N9 = 5 * 4 * 4


In [67]:
class ConvNet(nn.Module):
    
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 3, kernel_size=(3,1),stride=1) 
        
        self.pool1 = nn.MaxPool2d((3,1), stride=1)

        self.conv2 = nn.Conv2d(3, 5, kernel_size=(3,2),stride=1)

        self.drp1 = nn.Dropout2d(0.1)
        
        self.lin1 = nn.Linear(80, 7)
  
    def forward(self, x):
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.drp1(x)
        x = torch.relu(x)
        x = x.view(-1, 80)
        x = self.lin1(x)
        return x

In [68]:
cnn = ConvNet()
print(cnn)

ConvNet(
  (conv1): Conv2d(1, 3, kernel_size=(3, 1), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=(3, 1), stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(3, 5, kernel_size=(3, 2), stride=(1, 1))
  (drp1): Dropout2d(p=0.1, inplace=False)
  (lin1): Linear(in_features=80, out_features=7, bias=True)
)


# Treinamento

In [69]:
def evaluate(neural,x, y_hat):
  y = neural(x).argmax(dim=1)
  y_hat = y_hat.argmax(dim=1)
  return 100*float((y == y_hat).sum()) / len(y)

In [70]:
opt = optim.Adam(cnn.parameters(), lr=0.0001)
loss = nn.CrossEntropyLoss()

In [71]:
gpu = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cnn = cnn.to(gpu)
Xt = Xt.to(gpu, dtype=torch.float)
Yt = Yt.to(gpu, dtype=torch.long)
Xv = Xv.to(gpu, dtype=torch.float)
Yv = Yv.to(gpu, dtype=torch.long)

In [73]:
for j in range(2001):

  # Faremos o treinamento em lotes de
  # tamanho igual a 128 amostras
  for i in range(0,len(Yt),128):

    # Separa o lote de entradas
    x = Xt[i:i+128,:,:,:]

    # Separa o lote de saídas desejadas
    # já transformando de one-hot para
    # índice das colunas.
    y_hat = Yt[i:i+128,:].argmax(dim=1)

    # Zera o gradiente do otimizador
    opt.zero_grad()

    # Calcula a saída da rede neural
    y = cnn(x)

    # Calcula o erro
    e = loss(y, y_hat)

    # Calcula o gradiente usando
    # backpropagation
    e.backward()

    # Realiza um passo de atualização
    # dos parâmetros da rede neural
    # usando o otimizador.
    opt.step()

  # A cada 200 épocas imprimimos o
  # erro do último lote e a acurácia
  # nos dados de treinamento
  if not (j % 200):
    print(float(e), evaluate(cnn,Xt, Yt))

TypeError: evaluate() missing 1 required positional argument: 'y_hat'